# Huffman Coding – Homework

This notebook is your homework assignment on the topic of **Huffman coding**.




## Submission Instructions

Your final work must be submitted on **your personal GitHub account**, in a **public repository** dedicated to this assignment.

Your repository must include:

- the completed and cleanly executed notebook,
- any additional files you created for testing or analysis,
- a short `README.md` explaining how to run your notebook.

The deadline for submission is **December 15** (included).  
Any repository created or updated after this date will be considered late.



## Assignment Overview

Start by reading the wiki page on [Huffman coding](https://en.wikipedia.org/wiki/Huffman_coding) to understand the basic concepts.

**Objectives**:

- Compute character frequencies in a text.
- Build a Huffman tree using a min-heap.
- Derive binary codes from a Huffman tree.
- Implement encoding and decoding functions.

Read each text cell carefully, then complete the corresponding code cells.
You can run the test cells to check your work step by step.

## Exercise 1 – Character frequencies

We first need a function that counts how often each character appears in a text.

- The function should **ignore case**: treat `"A"` and `"a"` as the same character.
- It should return a **dictionary** whose keys are characters and whose values are their frequencies.

> Example:  
> `frequency("abA!")` could return `{ "a": 2, "b": 1, "!": 1 }` (the order of keys does not matter).


In [2]:
def frequency(text: str) -> dict[str, int]:
    """Return a dictionary mapping each character in `text` to its frequency.
    
    The function must:
    - convert the text to lowercase;
    - count how many times each character appears.
    """
    freq = {}
    text = text.lower()
    for char in text:
        freq[char] = freq.get(char, 0) +1

    return freq



In [3]:
# === Tests for Exercise 1 ===

assert frequency("") == {}
assert frequency("aaa") == {"a": 3}
assert frequency("abA!") == {"a": 2, "b": 1, "!": 1}

print("Exercise 1 seems OK (at least for these simple tests).")


Exercise 1 seems OK (at least for these simple tests).


## Exercise 2 – A node for the Huffman tree

We represent a Huffman tree using a `Node` class.

Each node must store:

- `characters`: a string containing all characters that appear in the subtree rooted at this node;
- `frequency`: the sum of the frequencies of these characters;
- `left`: the left child (another `Node`, or `None` for a leaf);
- `right`: the right child (another `Node`, or `None` for a leaf).

We will also define the method `__lt__` so that nodes can be compared by frequency when we use them in a **min-heap**.


In [4]:
class Node:
    def __init__(self, characters: str, frequency: int):
        """Create a new Huffman node.
        - 'characters' is the concatenation of all characters in this subtree.
        - 'frequency' is the total frequency of these characters.
        """
        self.characters = characters
        self.frequency = frequency
        self.left: Node | None = None
        self.right: Node | None = None

    def __lt__(self, other: "Node") -> bool:
        """Compare two nodes by frequency.
        This is needed so that we can use `heapq` on lists of nodes.
        """
        return self.frequency < other.frequency


In [5]:
# === Tests for Exercise 2 ===

n1 = Node("a", 5)
n2 = Node("b", 3)

# Check attributes exist
assert hasattr(n1, "characters")
assert hasattr(n1, "frequency")
assert hasattr(n1, "left")
assert hasattr(n1, "right")

# Check initial values
assert n1.characters == "a"
assert n1.frequency == 5
assert n1.left is None and n1.right is None

# Check comparison works (this must not crash)
nodes = [n1, n2]
from heapq import heapify
heapify(nodes)

print("Exercise 2 seems OK (basic tests passed).")


Exercise 2 seems OK (basic tests passed).


## Exercise 3 – Building the Huffman tree with a min-heap

We now build the Huffman tree from a dictionary of character frequencies.

**Algorithm (Huffman’s algorithm):**

1. Create a leaf `Node` for each character and push all these nodes into a **min-heap**.
2. While the heap contains more than one node:
   1. Pop the two nodes with the smallest frequency.
   2. Create a new internal node whose:
      - `left` child is the first node,
      - `right` child is the second node,
      - `characters` is the concatenation of the children’s `characters`,
      - `frequency` is the sum of the children’s `frequency`.
   3. Push this new node back into the heap.
3. The last remaining node in the heap is the **root of the Huffman tree**.


In [6]:
from heapq import heappush, heappop, heapify

def huffman_tree(freqs: dict[str, int]) -> Node:
    """Build the Huffman tree from a dictionary of frequencies.
    'freqs' maps characters to their frequencies.
    Return the root node of the Huffman tree.
    """
    heap = []

    for char, freq in freqs.items():
        heappush(heap, Node(char, freq))

    while len(heap) > 1:
        left = heappop(heap)
        right = heappop(heap)
        merged = Node(left.characters + right.characters, left.frequency + right.frequency)
        merged.left = left
        merged.right = right
        heappush(heap, merged)
    
    return heappop(heap)
        


In [7]:
# === Tests for Exercise 3 ===

freqs = frequency("this is an example for huffman encoding")
root = huffman_tree(freqs)

# Basic structural checks
assert isinstance(root, Node)
assert set(root.characters) == set(freqs.keys())
assert root.frequency == sum(freqs.values())

print("Exercise 3 seems OK (basic tests passed).")


Exercise 3 seems OK (basic tests passed).


## Exercise 4 – Code of a single character

We now want a function that returns the **binary code** of a given character,
based on the Huffman tree.

We assume that:

- a left edge corresponds to the bit `'0'`,
- a right edge corresponds to the bit `'1'`.

**Specification:**

- If `char` does **not** appear in the tree, raise a `ValueError`.
- Otherwise, return a string of `'0'` and `'1'` representing the path from the root to the leaf that contains `char`.
- You may implement this function **recursively**.


In [8]:
def get_code(tree: Node, char: str) -> str:
    """Return the Huffman code of 'char' in the given tree.
    Raise a ValueError if 'char' is not present in the tree.
    """
    if tree.left is None and tree.right is None:
        return ""
    
    elif tree.left and char in tree.left.characters:
        return "0" + get_code(tree.left, char)
    
    elif tree.right and char in tree.right.characters:
        return "1" + get_code(tree.right, char)
    
    else:
        raise ValueError(f"Character '{char}' not found in the tree.")
    



In [9]:
# === Tests for Exercise 4 ===

freqs = frequency("aaaabbc")
root = huffman_tree(freqs)

code_a = get_code(root, "a")
code_b = get_code(root, "b")
code_c = get_code(root, "c")

# Codes must be non-empty and different for different characters
assert isinstance(code_a, str) and len(code_a) > 0
assert isinstance(code_b, str) and len(code_b) > 0
assert isinstance(code_c, str) and len(code_c) > 0

assert code_a != code_b or code_a != code_c or code_b != code_c

print("Exercise 4 seems OK (basic tests passed).")


Exercise 4 seems OK (basic tests passed).


## Exercise 5 – Listing all codes

We now want to **print all Huffman codes** for all characters in the tree.

Complete the function `show_all_codes(tree)` so that it prints,
for each character in the tree, a line of the form:

```text
Character: x Code: 0101
```

You should reuse your function `get_code` from the previous exercise.


In [10]:
def show_all_codes(tree: Node) -> None:
    """Print the Huffman code of each character in the tree."""
    for char in tree.characters:
        code = get_code(tree, char)
        print(f"Character: {char} Code: {code}")


In [11]:
# === Manual test for Exercise 5 ===

freqs = frequency("this is an example for huffman encoding")
root = huffman_tree(freqs)

# This should print one line per distinct character
show_all_codes(root)


Character: s Code: 0000
Character: u Code: 00010
Character: d Code: 00011
Character: m Code: 0010
Character: h Code: 0011
Character: n Code: 010
Character: t Code: 01100
Character: l Code: 01101
Character: r Code: 01110
Character: g Code: 01111
Character: x Code: 10000
Character: p Code: 10001
Character: c Code: 10010
Character: o Code: 10011
Character: e Code: 1010
Character: a Code: 1011
Character: i Code: 1100
Character: f Code: 1101
Character:   Code: 111


## Exercise 6 – Encoding and decoding

We now implement the two main operations:

- `huffman_encode(text, tree)` returns a **string of bits** (a string of `'0'` and `'1'`),
- `huffman_decode(encoded_text, tree)` returns the original text.

### 6.1 – Encoding

To encode a text:

1. For each character `c` in `text`, look up its code with `get_code(tree, c)`.
2. Concatenate all these codes into a single string.

If a character of `text` does not appear in the tree, you may raise a `ValueError`.


In [12]:
def huffman_encode(text: str, tree: Node) -> str:
    """Return the Huffman encoding of 'text' using the given tree."""
    text = text.lower()
    encoded = ""

    for ch in text:
        encoded += get_code(tree, ch)

    return encoded

### 6.2 – Decoding

To decode a string of bits:

1. Start at the **root** of the tree.
2. For each bit in the encoded string:
   - if the bit is `'0'`, go to the **left** child;
   - if the bit is `'1'`, go to the **right** child.
3. Each time you reach a **leaf** (a node whose `left` and `right` are `None`):
   - append its character to the decoded text,
   - go back to the root.

At the end, you should have reconstructed the original text.


In [13]:
def huffman_decode(encoded_text: str, tree: Node) -> str:
    """Decode 'encoded_text' using the given Huffman tree."""
    decoded = ""
    node = tree

    for bit in encoded_text:
        if bit == "0":
            node = node.left
        else:
            node = node.right

        # Leaf node
        if node.left is None and node.right is None:
            decoded += node.characters
            node = tree

    return decoded


In [14]:
# === Tests for Exercise 6 ===

text = "this is an example for huffman encoding"
freqs = frequency(text)
root = huffman_tree(freqs)

encoded = huffman_encode(text, root)
decoded = huffman_decode(encoded, root)

print("Encoded text:", encoded[:64] + ("..." if len(encoded) > 64 else ""))
print("Decoded text:", decoded)

assert decoded == text.lower(), "Decoded text does not match original (lowercased)."

print("Exercise 6 seems OK (basic round-trip test passed).")


Encoded text: 0110000111100000011111000000111101101011110101000010110010100010...
Decoded text: this is an example for huffman encoding
Exercise 6 seems OK (basic round-trip test passed).


## Exercise 7 – A “universal” Huffman tree for English

So far, all Huffman trees you built were specialised to a specific text.  
In this exercise, you will construct a **general-purpose Huffman tree for English**.

For that purpose, we provide you with a dictionary `english_frequencies` that gives the approximate frequency of each character in English texts.

In [15]:
freqs_english = {
    " ": 18.0,
    "e": 12.02, "t": 9.10, "a": 8.12, "o": 7.68, "i": 7.31, "n": 6.95,
    "s": 6.28, "r": 6.02, "h": 5.92, "d": 4.32, "l": 3.98, "u": 2.88,
    "c": 2.71, "m": 2.61, "f": 2.30, "y": 2.11, "w": 2.09, "g": 2.03,
    "p": 1.82, "b": 1.49, "v": 1.11, "k": 0.69, "x": 0.17, "q": 0.11,
    "j": 0.10, "z": 0.07
}


Start by building a Huffman tree from this dictionary and test it by encoding and decoding the following text:

```text
"Huffman coding is a data compression algorithm." 
```

Make sure that the decoded text matches the original text!


In [ ]:
freqs = {
    " ": 18,
    "e": 12.02, "t": 9.10, "a": 8.12, "o": 7.68, "i": 7.31, "n": 6.95,
    "s": 6.28, "r": 6.02, "h": 5.92, "d": 4.32, "l": 3.98, "u": 2.88,
    "c": 2.71, "m": 2.61, "f": 2.30, "y": 2.11, "w": 2.09, "g": 2.03,
    "p": 1.82, "b": 1.49, "v": 1.11, "k": 0.69, "x": 0.17, "q": 0.11,
    "j": 0.10, "z": 0.07, ".": 0.5   
}

tree = huffman_tree(freqs)

#####

text = "Huffman coding is a data compression algorithm."
text = text.lower()

encoded_text = huffman_encode(text, tree)

print("Encoded text:")
print(encoded_text)

####

decoded_text = huffman_decode(encoded_text, tree)

print("Decoded text:")
print(decoded_text)

####

assert huffman_decode(encoded_text, tree) == text ## No issue, therefore it works


Encoded text:
000100001111101111101111111101001101100000010011011101110110101100110011101011101010110101111010111010101100000010011111111000010100001010101010111100101101101010100011011001001010001111110000111111110000011
Decoded text:
huffman coding is a data compression algorithm.


Compare the length of the encoded text with the length of the original text (in bits).You can assume that each character in the original text is represented using 8 bits (1 byte) using [ASCII encoding](https://en.wikipedia.org/wiki/ASCII).

In [ ]:
# TODO: Encode some sample texts and compare the length of the encoded text
# with the original text length in bits (assuming 8 bits per character)

english_tree = huffman_tree(freqs_english)

sample_text = "example"
encoded = huffman_encode(sample_text, english_tree)

original_bits = len(sample_text) * 8  # for the 8 bits
encoded_bits = len(encoded)

print("Original bits:", original_bits)
print("Encoded bits:", encoded_bits)
print("Compression ratio:", encoded_bits / original_bits)

Original bits: 56
Encoded bits: 37
Compression ratio: 0.6607142857142857
